Basic Statistics:

    Total Kills (K) per player. (By scope, by side, ) 
    Total Deaths (D) per player.
    Kill/Death ratio (K/D) per player.
    Average Combat Score (ACS) per player.
    Average Damage per Round (ADR) per player.

Performance Metrics:

    Headshot Percentage (HS%) per player.
    First Kills (FK) and First Deaths (FD) counts and ratios.
    Kill/Death difference (+/-) per player.
    Kill/Death difference in pistol rounds (+/-) per player.

Agent-Specific Metrics:

    Agent popularity and performance analysis.
    Win rates and performance on specific maps for each agent.

Team Performance:

    Total team Kills, Deaths, and K/D ratio.
    Team Average Combat Score (ACS) and Average Damage per Round (ADR).
    Team First Kills (FK) and First Deaths (FD) counts and ratios.

Map Performance:

    Individual player performance on each map.
    Team performance on each map.
    Win rates on each map.

Stage and Series Analysis:

    Performance in different stages (e.g., Playoffs, Grand Final).
    Series-level statistics and win rates.

Comparative Analysis:

    Compare individual player statistics against the team average.
    Compare agent performance across players.

Trend Analysis:

    Analyze how player performance changes over different maps or series.
    Identify trends in individual player K/D ratios.

Impact Analysis:

    Assess the impact of individual player performance on team success.


In [1]:
import pandas as pd
from utils import *

In [5]:
data = pd.read_csv('Data/stat_match_ascension.csv')
report = number_action_by_player_by_scope(df = data, action='K', average=True, team_name="Gentle Mates", side='Total')

In [1]:
import pandas as pd
from utils import *

df = pd.read_csv('champions-tour-2024-pacific-kickoff_data/general_data_champions-tour-2024-pacific-kickoff.csv')



In [2]:
win_r = calculate_win_rate(df)

In [3]:
win_r

{'BLEED': 0.0,
 'ZETA DIVISION': 0.5,
 'T1': 0.67,
 'DRX': 0.67,
 'Global Esports': 0.33,
 'Paper Rex': 0.67,
 'Rex Regum Qeon': 0.0,
 'Team Secret': 0.5,
 'DetonatioN FocusMe': 0.0,
 'Talon Esports': 0.33,
 'Gen.G': 0.83}